<a href="https://colab.research.google.com/github/nicolemichaud03/Recipe-Recommender-System/blob/main/Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capstone Project - Recipe/Meal Recommendation System

In [1]:
!pip install scikit-surprise
# !conda install -y -c conda-forge scikit-surprise # If you use conda on a non-Colab environment



In [2]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from surprise.model_selection import train_test_split
import io
import surprise
from surprise import SVD, BaselineOnly, SVDpp, NMF
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading the user and item data and viewing it:

In [4]:
user_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_interactions.csv')

In [5]:
recipe_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_recipes.csv')

In [6]:
user_data.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [7]:
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [8]:
#dropping columns any additional columns to prepare df for Suprise (can only take in user_id, item_id, and ratings)
recipe_df = recipe_data.drop(columns=['contributor_id', 'submitted', 'tags', 'nutrition', 'description', 'ingredients', 'n_steps', 'steps', 'n_ingredients'])
recipe_df.head()

,name,id,minutes
0,arriba baked winter squash mexican style,137739,55
1,a bit different breakfast pizza,31490,30
2,all in the kitchen chili,112140,130
3,alouette potatoes,59389,45
4,amish tomato ketchup for canning,44061,190


In [9]:
# read in values as Surprise dataset
from surprise import Reader, Dataset

reader = Reader()
recipe_df = Dataset.load_from_df(recipe_df, reader)


In [10]:
print(recipe_df)

In [11]:
user_df = user_data.drop(columns=['date', 'review'])
user_df.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [12]:
user_df = Dataset.load_from_df(user_df, reader)
user_df

In [13]:
#additional preprocessed dataframes from original source, don't know if I will use these
#recipes_PP = pd.read_csv('data/PP_recipes.csv')
#users_PP = pd.read_csv('data/PP_users.csv')

In [14]:
#pickled_map = pd.read_pickle('data/ingr_map.pkl')
#pickled_map

## Data Exploration

In [15]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [16]:
print(len(user_data['user_id'].unique()))

226570


There are 226,570 unique users in this dataset

In [17]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [18]:
recipe_data = recipe_data.drop(columns=['contributor_id', 'submitted', 'nutrition', 'steps', 'minutes', 'n_steps', 'n_ingredients'])
recipe_data = recipe_data.rename(columns={"id": "recipe_id"})

In [19]:
print(len(recipe_data['recipe_id'].unique()))

231637


There are 231,637 unique recipes in this dataset.

With 226,570 users and 231,637 recipes, there are less users than there are recipes. Therefore, it is probably best for our recommender system to be user-user based.

## Preprocessing

I'm going to be using text data from the 'ingredients' and 'tags' features to classify recipes and users as different diet types. In my miscellaneous notebook I show why I chose these two features.

The 'ingredients' feature is already pretty well cleaned, because it is just a list of basic ingredient words. The 'tags' need to be cleaned though.

In [20]:
# Creating a function to perform cleaning steps at once
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    #text = no_nums.sub('', text)
    text = no_bad_chars.sub(' ', text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
tokenized_text_data = recipe_data.copy()

In [22]:
tokenized_text_data['description'] = tokenized_text_data['description'].astype(str)
tokenized_text_data['name'] = tokenized_text_data['name'].astype(str)

In [23]:
tokenized_text_data['tags'] = tokenized_text_data['tags'].apply(clean_text)
tokenized_text_data['description'] = tokenized_text_data['description'].apply(clean_text)
tokenized_text_data['name'] = tokenized_text_data['name'].apply(clean_text)
tokenized_text_data.head()

,name,recipe_id,tags,description,ingredients
0,arriba baked winter squash mexican style,137739,'60 minutes less' 'time make' 'course' 'main i...,autumn favorite time year cook recipe prepared...,"['winter squash', 'mexican seasoning', 'mixed ..."
1,bit different breakfast pizza,31490,'30 minutes less' 'time make' 'course' 'main i...,recipe calls crust prebaked bit adding ingredi...,"['prepared pizza crust', 'sausage patty', 'egg..."
2,kitchen chili,112140,'time make' 'course' 'preparation' 'main dish'...,modified version 'mom's' chili hit 2004 christ...,"['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,59389,'60 minutes less' 'time make' 'course' 'main i...,super easy great tasting make ahead side dish ...,"['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup canning,44061,'weeknight' 'time make' 'course' 'main ingredi...,dh's amish mother raised recipe much prefers s...,"['tomato juice', 'apple cider vinegar', 'sugar..."


Creating features for whether or not each recipe is Gluten-Free, Vegetarian, or Vegan. I then use these features to classify users as these diet types if at least 75% of the recipes they have interacted with fall in any of these categories.

In [24]:
GF = []
for row in recipe_data['tags']:
    if "gluten-free" in row : GF.append(1)
    elif "gluten free" in row : GF.append(1)
    else: GF.append(0)

In [25]:
recipe_data['GF'] = GF

In [26]:
recipe_data['GF'].value_counts()

0    225894
1      5743
Name: GF, dtype: int64

In [27]:
#Ingredient lists for diet filtering:
vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']

In [28]:
recipe_data['vegetarian'] = None
recipe_data['vegan'] = None

In [29]:
vege_pattern = '|'.join(vegetarian)
vegan_pattern = '|'.join(vegan)


recipe_data.vegetarian = recipe_data.ingredients.str.contains(vege_pattern)
recipe_data.vegan = recipe_data.ingredients.str.contains(vegan_pattern)

In [30]:
recipe_data.head()

,name,recipe_id,tags,description,ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,"['60-minutes-or-less', 'time-to-make', 'course...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",0,False,True
1,a bit different breakfast pizza,31490,"['30-minutes-or-less', 'time-to-make', 'course...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",0,True,True
2,all in the kitchen chili,112140,"['time-to-make', 'course', 'preparation', 'mai...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",0,True,True
3,alouette potatoes,59389,"['60-minutes-or-less', 'time-to-make', 'course...","this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",0,False,True
4,amish tomato ketchup for canning,44061,"['weeknight', 'time-to-make', 'course', 'main-...",my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",0,False,False


In [31]:
#new_vals = {'False': 1, 'True': 0}

In [32]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   name         231636 non-null  object
 1   recipe_id    231637 non-null  int64 
 2   tags         231637 non-null  object
 3   description  226658 non-null  object
 4   ingredients  231637 non-null  object
 5   GF           231637 non-null  int64 
 6   vegetarian   231637 non-null  bool  
 7   vegan        231637 non-null  bool  
dtypes: bool(2), int64(2), object(4)
memory usage: 11.0+ MB


In [33]:
recipe_data['vegetarian'] = recipe_data['vegetarian'].astype(str)
recipe_data['vegetarian'] = recipe_data['vegetarian'].replace({'False': 1, 'True': 0})

In [34]:
recipe_data['vegan'] = recipe_data['vegan'].astype(str)
recipe_data['vegan'] = recipe_data['vegan'].replace({'False': 1, 'True': 0})

In [35]:
recipe_data.head()

,name,recipe_id,tags,description,ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,"['60-minutes-or-less', 'time-to-make', 'course...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",0,1,0
1,a bit different breakfast pizza,31490,"['30-minutes-or-less', 'time-to-make', 'course...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",0,0,0
2,all in the kitchen chili,112140,"['time-to-make', 'course', 'preparation', 'mai...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",0,0,0
3,alouette potatoes,59389,"['60-minutes-or-less', 'time-to-make', 'course...","this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",0,1,0
4,amish tomato ketchup for canning,44061,"['weeknight', 'time-to-make', 'course', 'main-...",my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",0,1,1


In [36]:
recipe_data['vegetarian'].value_counts()

1    134596
0     97041
Name: vegetarian, dtype: int64

In [37]:
recipe_data['vegan'].value_counts()

0    197961
1     33676
Name: vegan, dtype: int64

In [38]:
#Making column names match and merging dfs to classify user diets based on the recipes they've used
recipe_data = recipe_data.rename(columns = {'id': 'recipe_id'})

user_diets = pd.merge(user_data, recipe_data, on='recipe_id', how='left')



In [39]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132367 entries, 0 to 1132366
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   user_id      1132367 non-null  int64 
 1   recipe_id    1132367 non-null  int64 
 2   date         1132367 non-null  object
 3   rating       1132367 non-null  int64 
 4   review       1132198 non-null  object
 5   name         1132366 non-null  object
 6   tags         1132367 non-null  object
 7   description  1108857 non-null  object
 8   ingredients  1132367 non-null  object
 9   GF           1132367 non-null  int64 
 10  vegetarian   1132367 non-null  int64 
 11  vegan        1132367 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 112.3+ MB


In [40]:
len(user_diets['user_id'].unique())

226570

In [41]:
user_diets = (user_diets.groupby(['user_id']).agg({'recipe_id': lambda x: x.tolist(), 'rating': lambda x: x.tolist(), 'GF': sum, 'vegetarian':sum, 'vegan': sum}).reset_index())
user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",20,630,114
2,1581,[341050],[5],0,0,0
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...","[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",3,35,9
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...","[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",3,4,2


In [42]:
user_diets['is_vegetarian'] = None
user_diets['is_vegan'] = None
user_diets['is_GF'] = None
user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,None,None,None
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",20,630,114,None,None,None
2,1581,[341050],[5],0,0,0,None,None,None
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...","[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",3,35,9,None,None,None
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...","[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",3,4,2,None,None,None


In [43]:
user_diets['recipe_totals'] = user_diets['recipe_id'].str.len()

In [44]:
user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF,recipe_totals
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,None,None,None,128
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",20,630,114,None,None,None,794
2,1581,[341050],[5],0,0,0,None,None,None,1
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...","[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",3,35,9,None,None,None,60
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...","[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",3,4,2,None,None,None,31


In [45]:
user_diets['is_vegetarian'] = np.where(user_diets['vegetarian'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)

user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF,recipe_totals
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,0,None,None,128
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",20,630,114,1,None,None,794
2,1581,[341050],[5],0,0,0,0,None,None,1
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...","[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",3,35,9,0,None,None,60
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...","[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",3,4,2,0,None,None,31


In [46]:
user_diets['is_vegetarian'].value_counts()

1    114857
0    111713
Name: is_vegetarian, dtype: int64

In [47]:
user_diets['is_vegan'] = np.where(user_diets['vegan'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)
user_diets['is_vegan'].value_counts()

0    206565
1     20005
Name: is_vegan, dtype: int64

In [48]:
user_diets['is_GF'] = np.where(user_diets['GF'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)
user_diets['is_GF'].value_counts()

0    220978
1      5592
Name: is_GF, dtype: int64

In [49]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226570 entries, 0 to 226569
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        226570 non-null  int64 
 1   recipe_id      226570 non-null  object
 2   rating         226570 non-null  object
 3   GF             226570 non-null  int64 
 4   vegetarian     226570 non-null  int64 
 5   vegan          226570 non-null  int64 
 6   is_vegetarian  226570 non-null  int64 
 7   is_vegan       226570 non-null  int64 
 8   is_GF          226570 non-null  int64 
 9   recipe_totals  226570 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 17.3+ MB


Attempting to make separate user and item df's that also include the extra diet features (If I found a way to create and multiply in additional matrices with user features and item features, I would need these separate and in a way that is usable)

In [50]:
user_diets_only = user_diets[['user_id', 'recipe_id', 'rating', 'is_GF', 'is_vegetarian', 'is_vegan']]
recipe_diets = user_diets[[ 'recipe_id', 'rating', 'GF', 'vegetarian', 'vegan']]

In [51]:
user_diets_only.head()

,user_id,recipe_id,rating,is_GF,is_vegetarian,is_vegan
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,0,0
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,1,0
2,1581,[341050],[5],0,0,0
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...","[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",0,0,0
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...","[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",0,0,0


In [52]:

#user_diets_only = user_diets_only.explode(['user_id','recipe_id', 'rating'])
#user_diets_only.head()

In [53]:
#user_diets = user_diets.drop(columns=['date', 'review', 'name', 'minutes', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients'])
#user_diets.head()

In [54]:
user_diets_full = user_diets.explode('recipe_id')

In [55]:
user_diets_full.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF,recipe_totals
0,1533,116345,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,0,0,0,128
0,1533,32907,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,0,0,0,128
0,1533,14750,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,0,0,0,128
0,1533,24136,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,0,0,0,128
0,1533,63598,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8,0,0,0,128


In [56]:
user_features = user_diets_full.drop(columns=['GF', 'vegetarian', 'vegan', 'recipe_totals'])
user_features.head()

,user_id,recipe_id,rating,is_vegetarian,is_vegan,is_GF
0,1533,116345,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,0,0
0,1533,32907,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,0,0
0,1533,14750,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,0,0
0,1533,24136,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,0,0
0,1533,63598,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,0,0


In [57]:
recipe_data

,name,recipe_id,tags,description,ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,"['60-minutes-or-less', 'time-to-make', 'course...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",0,1,0
1,a bit different breakfast pizza,31490,"['30-minutes-or-less', 'time-to-make', 'course...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",0,0,0
2,all in the kitchen chili,112140,"['time-to-make', 'course', 'preparation', 'mai...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",0,0,0
3,alouette potatoes,59389,"['60-minutes-or-less', 'time-to-make', 'course...","this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",0,1,0
4,amish tomato ketchup for canning,44061,"['weeknight', 'time-to-make', 'course', 'main-...",my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",0,1,1
...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,"['ham', '60-minutes-or-less', 'time-to-make', ...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",0,0,0
231633,zydeco spice mix,493372,"['15-minutes-or-less', 'time-to-make', 'course...",this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",0,1,1
231634,zydeco ya ya deviled eggs,308080,"['60-minutes-or-less', 'time-to-make', 'course...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",0,1,0
231635,cookies by design cookies on a stick,298512,"['30-minutes-or-less', 'time-to-make', 'course...","i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",0,1,0


In [58]:
#Haven't found out how to explode the ratings values as well...
item_features = recipe_data.drop(columns=['name', 'description', 'tags', 'ingredients'])
item_features.head()

,recipe_id,GF,vegetarian,vegan
0,137739,0,1,0
1,31490,0,0,0
2,112140,0,0,0
3,59389,0,1,0
4,44061,0,1,1


In [59]:
user_features['is_vegetarian'] = user_features['is_vegetarian'].astype(str)
user_features['is_vegetarian'] = user_features['is_vegetarian'].replace({"1": "vegetarian", "0": "none"})
user_features['is_vegan'] = user_features['is_vegan'].astype(str)
user_features['is_vegan'] = user_features['is_vegan'].replace({"1": "vegetarian", "0": "none"})
user_features['is_GF'] = user_features['is_GF'].astype(str)
user_features['is_GF'] = user_features['is_GF'].replace({"1": "vegetarian", "0": "none"})


In [60]:
user_features = user_features.drop_duplicates(['user_id'])

In [61]:
user_features

,user_id,recipe_id,rating,is_vegetarian,is_vegan,is_GF
0,1533,116345,"[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",none,none,none
1,1535,349022,"[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",vegetarian,none,none
2,1581,341050,[5],none,none,none
3,1634,16512,"[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",none,none,none
4,1676,34233,"[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",none,none,none
...,...,...,...,...,...,...
226565,2002371755,393600,[5],vegetarian,none,none
226566,2002371792,431399,[4],vegetarian,none,none
226567,2002371843,516662,[5],vegetarian,none,none
226568,2002372464,491512,[4],vegetarian,none,none


In [62]:
item_features['vegetarian'] = item_features['vegetarian'].astype(str)
item_features['vegetarian'] = item_features['vegetarian'].replace({"1": "vegetarian", "0": "none"})
item_features['vegan'] = item_features['vegan'].astype(str)
item_features['vegan'] = item_features['vegan'].replace({"1": "vegan", "0": "none"})
item_features['GF'] = item_features['GF'].astype(str)
item_features['GF'] = item_features['GF'].replace({"1": "GF", "0": "none"})

In [63]:
item_features

,recipe_id,GF,vegetarian,vegan
0,137739,none,vegetarian,none
1,31490,none,none,none
2,112140,none,none,none
3,59389,none,vegetarian,none
4,44061,none,vegetarian,vegan
...,...,...,...,...
231632,486161,none,none,none
231633,493372,none,vegetarian,vegan
231634,308080,none,vegetarian,none
231635,298512,none,vegetarian,none


## Modeling

Creating recipe_diet_df, that only contains the each recipe ID and if the recipe is vegan, vegetarian, and/or gluten-free, and user_diet_df that only contains the each user ID and if the user is vegan, vegetarian, and/or gluten-free:

In [ ]:
recipe_diet_df = item_features.copy()
recipe_diet_df['diet'] = recipe_diet_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

user_diet_df = user_features.copy()
user_diet_df['diet'] = user_diet_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

In [ ]:
recipe_diet_df

In [ ]:
#count vectorize features of diet type to be used to create a matrix of item features:

from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()
item_matrix = count_vec.fit_transform(recipe_diet_df['diet'])



In [ ]:
item_matrix

In [ ]:
#1
from sklearn.metrics.pairwise import linear_kernel
content_similarity = linear_kernel(item_matrix, item_matrix)

#reader = Reader(rating_scale=(1, 5))
#data = Dataset.load_from_df(data[['User ID',
                                  #'Product ID',
                                  #'Rating']], reader)

def get_content_based_recommendations(recipe_id, top_n):
    index = recipe_diet_df[recipe_diet_df['recipe_id'] == recipe_id].index[0]
    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendations = recipe_diet_df.loc[similar_indices, 'recipe_id'].values
    return recommendations

In [ ]:
def get_content_based_recommendations(recipe_id, top_n):
    try:
        index = recipe_diet_df[recipe_diet_df['recipe_id'] == recipe_id].index[0]
    except IndexError:
        # Recipe ID not found, return an empty list
        return []

    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    return similar_indices

In [ ]:
content_based_recommendations = get_content_based_recommendations(recipe_diet_df['recipe_id'], 3)

In [ ]:
content_based_recommendations

In [69]:
#Collaborative filtering 1
algo1 = SVD()
trainset = user_df.build_full_trainset()
algo1.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions1 = algo1.test(testset)
    predictions1.sort(key=lambda x: x.est, reverse=True)
    recommendations1 = [prediction.iid for prediction in predictions1[:top_n]]
    return recommendations1

In [70]:
#hybrid 1
# (user_ratings,movie_title_df,n)
# for idx, rec in enumerate(user_ratings):
def get_hybrid_recommendations(user_data, recipe_data, top_n):
    for user_id in user_data:
      for recipe_id in recipe_data:
        content_based_recommendations = get_content_based_recommendations(recipe_id, top_n)
        collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
        hybrid_recommendations = list(set(content_based_recommendations + collaborative_filtering_recommendations))
        return hybrid_recommendations[:top_n]

In [74]:
get_hybrid_recommendations(user_data, recipe_data, 3)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [71]:
#trying to adapt this code for our hybrid model (since we know this works)
#from surprise documentation: https://surprise.readthedocs.io/en/stable/FAQ.html
def get_top_n(predictions, n=3):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the dataset.
data = Dataset.load_from_df(user_data, reader)
trainset = data.build_full_trainset()
algo_ = SVD(n_factors=20, n_epochs=10, reg_all=0.05)
algo_.fit(trainset)



# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

ValueError: too many values to unpack (expected 3)

In [75]:
print(accuracy.rmse(get_hybrid_recommendations))

TypeError: 'function' object is not iterable

In [72]:
np.mean(algo1['test_rmse'])

TypeError: 'SVD' object is not subscriptable

_______

If above attempt doesnt work on vectorized diet columns, attempting to use the text features of the recipes for a hybrid model:

In [ ]:
# Combining content-based and CF methods, content-based filtering first:2
#https://thecleverprogrammer.com/2023/06/05/hybrid-recommendation-system-using-python/
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# recipe_text_df = recipe_data.loc['recipe_id', 'name', 'description',
#                    'ingredients', 'tags']
# recipe_text_df['Content'] = recipe_text_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Use TF-IDF vectorizer to convert content into a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(recipe_text_df['Content'])

content_similarity = linear_kernel(content_matrix, content_matrix)

#reader = Reader(rating_scale=(1, 5))
#data = Dataset.load_from_df(data[['User ID',
                                  #'Product ID',
                                  #'Rating']], reader)

def get_content_based_recommendations(product_id, top_n):
    index = content_df[content_df['Product ID'] == product_id].index[0]
    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendations = content_df.loc[similar_indices, 'Product ID'].values
    return recommendations

<ipython-input-58-abd34a15517e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_text_df['Content'] = recipe_text_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)


In [ ]:
#Collaborative filtering
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = algo.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations

In [ ]:
#hybrid
def get_hybrid_recommendations(user_id, product_id, top_n):
    content_based_recommendations = get_content_based_recommendations(product_id, top_n)
    collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
    hybrid_recommendations = list(set(content_based_recommendations + collaborative_filtering_recommendations))
    return hybrid_recommendations[:top_n]

________

Trying out matrix multiplication method on a small subset of the data to see if it is possible:

In [79]:
user_features = user_features.explode(['rating'])
user_features.head()

,user_id,recipe_id,rating,is_vegetarian,is_vegan,is_GF
0,1533,116345,5,none,none,none
0,1533,116345,5,none,none,none
0,1533,116345,5,none,none,none
0,1533,116345,5,none,none,none
0,1533,116345,4,none,none,none


In [80]:
item_features

,recipe_id,GF,vegetarian,vegan
0,137739,none,vegetarian,none
1,31490,none,none,none
2,112140,none,none,none
3,59389,none,vegetarian,none
4,44061,none,vegetarian,vegan
...,...,...,...,...
231632,486161,none,none,none
231633,493372,none,vegetarian,vegan
231634,308080,none,vegetarian,none
231635,298512,none,vegetarian,none


In [82]:
#user_features['recipe_id'].compare(item_features['recipe_id'], keep_equal=True)

In [84]:
merged_user_recipes = user_features.merge(item_features, on="recipe_id", how="inner")

In [85]:
merged_user_recipes

,user_id,recipe_id,rating,is_vegetarian,is_vegan,is_GF,GF,vegetarian,vegan
0,1533,116345,5,none,none,none,none,vegetarian,none
1,1533,116345,5,none,none,none,none,vegetarian,none
2,1533,116345,5,none,none,none,none,vegetarian,none
3,1533,116345,5,none,none,none,none,vegetarian,none
4,1533,116345,4,none,none,none,none,vegetarian,none
...,...,...,...,...,...,...,...,...,...
1132362,2002369939,528696,0,none,none,none,none,none,none
1132363,2002370136,397552,0,none,none,none,none,none,none
1132364,2002370648,356948,3,vegetarian,none,none,none,vegetarian,none
1132365,2002371712,144174,5,none,none,none,none,none,none


In [87]:
test_subset = merged_user_recipes.iloc[:25]
test_subset.head()


,user_id,recipe_id,rating,is_vegetarian,is_vegan,is_GF,GF,vegetarian,vegan
0,1533,116345,5,none,none,none,none,vegetarian,none
1,1533,116345,5,none,none,none,none,vegetarian,none
2,1533,116345,5,none,none,none,none,vegetarian,none
3,1533,116345,5,none,none,none,none,vegetarian,none
4,1533,116345,4,none,none,none,none,vegetarian,none


In [76]:
test_subset = test_subset.explode(['recipe_id', 'rating', 'GF', 'vegetarian', 'vegan'])
test_subset.head()

ValueError: columns must have matching element counts

In [73]:
test_subset['is_vegetarian'].value_counts()

1    813
0    305
Name: is_vegetarian, dtype: int64

In [71]:
#train-test splitting the data subset
trainset, testset = train_test_split(test_subset, test_size=0.25)

AttributeError: 'DataFrame' object has no attribute 'raw_ratings'

In [ ]:
svd = SVD(n_factors=10, n_epochs=10, lr_all=0.005, reg_all=0.07, random_state=42)
#n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4
svd.fit(trainset)
preds_svd = svd.test(testset)
print(accuracy.rmse(preds_svd))

____________

Seeing what model works best for this data:
(Commented out due to long runtime)

In [57]:
# #Iterate over all potential algorithms to see which one performs best

# benchmark = []


# algorithms = [SVD(), SVDpp(), NMF(), BaselineOnly()]

# print ("Attempting: ", str(algorithms), '\n\n\n')

# for algorithm in algorithms:
#     #print("Starting: " ,str(algorithm))
#     # Perform cross validation
#     results = cross_validate(algorithm, user_df, measures=['RMSE'], cv=3, verbose=False)
#     # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)

#     # Get results & append algorithm name
#     tmp = pd.DataFrame.from_dict(results).mean(axis=0)
#     tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
#     benchmark.append(tmp)
#     print("Done: " ,str(algorithm), "\n\n")

# print ('\n\tDONE\n')

In [58]:
#surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [59]:
#surprise_results

In [60]:
#results_df = pd.DataFrame.from_dict(BaselineOnly.cv_results)

It looks like the baseline only model performed best and took the least amount of time. SVD was second best. Can we improve the SVD model with parameter tuning?

(Commented out due to long runtime)

In [61]:
# params = {'n_factors': [10,20,50],
#          'reg_all': [ 0.02, 0.05, 0.07]}
# g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)
# g_s_svd.fit(user_df)

In [62]:
# # print out optimal parameters for SVD after GridSearch
# print(g_s_svd.best_score)
# print(g_s_svd.best_params)

Best parameters for SVD were n_factors=10 and reg_all=0.05. The RMSE of this model is slightly better than for the BaselineOnly model, so it is currently our best model.

In [65]:
#train-test split the data

trainset, testset = train_test_split(user_df, test_size=0.25)

In [86]:
#Basic SVD model without any extra features, but with these tuned parameters:

algo = SVD()
preds = algo.fit(trainset).test(testset)

#surprise.accuracy.rmse(preds, verbose=True)

#algo_normal_predictor = NormalPredictor()
#predictions = algo_normal_predictor.fit(trainset).test(testset)

(Also try without b)

In [87]:
svd_pred = cross_validate(SVD(), user_df, verbose=True)
print(np.mean(svd_pred['test_rmse']))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2145  1.2274  1.2173  1.2214  1.2196  1.2201  0.0044  
MAE (testset)     0.7373  0.7439  0.7374  0.7402  0.7389  0.7396  0.0024  
Fit time          30.98   36.05   37.77   36.69   37.82   35.86   2.53    
Test time         2.75    4.75    2.21    1.89    3.04    2.93    1.00    
1.2200544406794491


In [64]:
# # This function to generate predictions if only using basic SVD model
# #from surprise documentation: https://surprise.readthedocs.io/en/stable/FAQ.html
# def get_top_n(predictions, n=3):


#     # First map the predictions to each user.
#     top_n = defaultdict(list)
#     for uid, iid, true_r, est, _ in predictions:
#         top_n[uid].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
#     for uid, user_ratings in top_n.items():
#         user_ratings.sort(key=lambda x: x[1], reverse=True)
#         top_n[uid] = user_ratings[:n]

#     return top_n


# # First train an SVD algorithm on the dataset.
# data = Dataset.load_from_df(user_data, reader)
# trainset = data.build_full_trainset()
# algo_ = SVD(n_factors=20, n_epochs=10, reg_all=0.05)
# algo_.fit(trainset)



# # Than predict ratings for all pairs (u, i) that are NOT in the training set.
# testset = trainset.build_anti_testset()
# predictions = algo.test(testset)


The equation for the surprise SVD algorithm is: Rui = u + bu + bi + qi^T*pu
(The matrix of users, items, and ratings is decomposed into the unknown user "u", plus the bias of the user, plus the bias of the item, plus the transpose matrix of item factors times the matrix of user factors)

My assumption is if you could successfully incorporate a matrix of additional user features (multiply it with) in the user factors matrix (pu), and an additional matrix of item features in the item features matrix (qi), and re-add all the aspects of this equation together, performing SVD on that resulting matrix would take into account extra user and item features that the original matrix did not. I do not yet understand quite how to do this, though.

In [89]:
#Prints out the resulting 'Item Factors' Matrix
algo.qi

array([[ 0.10899472,  0.03561716,  0.09309703, ..., -0.19734205,
         0.0813999 ,  0.06711077],
       [-0.1962503 , -0.14126231,  0.22689873, ..., -0.05153303,
         0.09093677, -0.04340261],
       [ 0.16681218, -0.09884306,  0.00614764, ..., -0.02431735,
        -0.03950759, -0.08065413],
       ...,
       [-0.01601718, -0.02547537, -0.01349926, ..., -0.07778163,
         0.0413141 ,  0.05196922],
       [-0.07756209,  0.13706307, -0.09535434, ..., -0.08187619,
         0.09772491,  0.04579227],
       [-0.03799774,  0.18765242, -0.02317286, ...,  0.27137191,
        -0.06233448, -0.0065714 ]])

In [90]:
#Prints out the resulting 'Item Factors' Matrix
algo.pu

array([[-0.16591695, -0.11492225, -0.04650306, ..., -0.01898454,
         0.17048896, -0.10684066],
       [ 0.01112167,  0.23237592,  0.0618511 , ...,  0.17708566,
         0.07245186, -0.04453099],
       [-0.07353296,  0.08217174,  0.11302679, ..., -0.04790352,
         0.18818894,  0.24925114],
       ...,
       [-0.01609043,  0.03023127,  0.28667628, ..., -0.00193458,
         0.07812412,  0.07631334],
       [ 0.1178472 ,  0.06553045, -0.07182736, ...,  0.03963503,
         0.08719834,  0.08039509],
       [-0.0152722 ,  0.22419084,  0.20691753, ..., -0.13720106,
        -0.06492597, -0.13340213]])

______

In [76]:
##initial SVD without features: (computer crashes, doesnt work)
##I was trying to do SVD 'from scratch' to add in additional features but this was unsuccessful

#from numpy import array
#from scipy.linalg import svd
## define a matrix
#A = ratings_matrix

# SVD
#U, s, VT = svd(ratings_matrix)
#print(U)
#print(s)
#print(VT)

In [77]:
#attempted to combine matrices for using additional user/item features
#This did not work

#print('Rounded Approximation of Ratings Matrix')
#np.round(ratings_matrix.dot(np.user_features_matrix(s).item_features_matrix(vt)))

In [79]:
#same as above code block, slightly different approach. Still didn't work

#matrix_A = ratings_matrix.dot(user_features_matrix).dot(item_features_matrix)
#A.dot(B).dot(C)

In [83]:
# def run_svd(self):
#         u, s, vt = svds(self.training_csc, k=self.k)

#         self.Q = u
#         diag_matrix = np.diag(s)
#         self.P = diag_matrix.dot(vt)

The following is data processing meant for a version of SVD that was produced by AI and did not end up being mathmaticaly correct, but its possible I can use some of the steps for something else:

In [68]:
user_item_ratings = user_data[['user_id', 'recipe_id', 'rating']]
user_item_ratings.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [69]:
users_df = user_data[['user_id', 'recipe_id', 'rating']]
users_df.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [70]:
items_df = recipe_data
items_df.head()

,name,recipe_id,minutes,tags,n_steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,1,0
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,0,0
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,0,0
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,1,0
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,1,1


In [71]:
user_features_df = user_features.copy()

In [72]:
item_features_df = recipe_data[['recipe_id', 'vegetarian', 'vegan', 'GF']]

In [73]:
#for simplicity, renaming column 'recipe_id' to 'item_id'
items_df = items_df.copy().rename(columns={'recipe_id': 'item_id'})
item_features_df = item_features_df.copy().rename(columns={'recipe_id': 'item_id'})
user_item_ratings = user_item_ratings.copy().rename(columns={'recipe_id': 'item_id'})
users_df = user_item_ratings.copy().rename(columns={'recipe_id': 'item_id'})


In [74]:
#This is an alternative way to generate the necessary matrices - doing so by pivoting my df caused the new data to be too large for my computer

# Create a mapping between user/item IDs and matrix indices
user_id_to_index = {user_id: index for index, user_id in enumerate(users_df['user_id'])}
item_id_to_index = {item_id: index for index, item_id in enumerate(items_df['item_id'])}

In [75]:

# Initialize the ratings matrix
ratings_matrix = np.zeros((len(users_df), len(items_df)))

# Fill in the ratings matrix based on user_item_ratings DataFrame
for _, row in users_df.iterrows():
  user_index = user_id_to_index.get(row['user_id'])
  item_index = item_id_to_index.get(row['item_id'])
  if user_index is not None and item_index is not None:
    ratings_matrix[user_index, item_index] = row['rating']



In [80]:
#define variables for class/function later on
num_users = len(set(user_item_ratings))
num_items = len(recipe_data)
num_user_features = len(user_features_df.columns) - 1  # Exclude user_id
num_item_features = len(item_features_df.columns) - 1  # Exclude item_id


In [81]:
# creating the matrices for user and item features for the edited SVD algorithm that didn't work
# all these steps up until the actual SVD step worked, though

user_features_matrix = np.zeros((len(user_features_df), num_user_features))
item_features_matrix = np.zeros((len(item_features_df), num_item_features))

for _, row in user_features.iterrows():
   user_index = user_features.get(row['user_id'])
   if user_index is not None:
    user_features_matrix[user_index, :] = row.values[1:] # Assuming user features start from the second column

for _, row in item_features_df.iterrows():
    item_index = items_df.get(row['item_id'])
    if item_index is not None:
        item_features_matrix[item_index, :] = row.values[1:]  # Assuming item features start from the second column


In [82]:
##This was from troubleshooting a previous issue
#user_features_matrix = user_features.iloc[:, 1:].values  # Assuming user features start from the second column
#item_features_matrix = item_features_df.iloc[:, 1:].values  # Assuming item features start from the second column


In [ ]:
#this cell and the next are 2 slightly different versions of the AI produced SVD code,
#to my knowledge the math is not exactly right but it is too complex to find out where it is wrong,
# and not much documentation on how this would actually look for SVD since it is usually automatic through a library (surprise, scipy, etc.)

class SVDModel:
    def __init__(self, num_users, num_items, num_user_features, num_item_features, n_factors=10, reg_all=0.07, alpha=0.01, beta=0.01, iterations=1000):
        self.num_users = num_users
        self.num_items = num_items
        self.num_user_features = num_user_features
        self.num_item_features = num_item_features
        self.n_factors = n_factors
        self.reg_all = reg_all
        self.alpha = alpha  # Regularization term for user and item matrices
        self.beta = beta    # Learning rate
        self.iterations = iterations

    def fit(self, ratings, user_features, item_features):
        # Initialize user and item matrices
        #self.user_matrix = np.random.rand(self.num_users, self.n_factors)
        #self.item_matrix = np.random.rand(self.num_items, self.n_factors)

        # Initialize user and item feature matrices
        self.user_feature_matrix = np.random.rand(self.num_users, self.num_user_features)
        self.item_feature_matrix = np.random.rand(self.num_items, self.num_item_features)

        # Gradient descent for SVD model with features
        for _ in range(self.iterations):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if ratings[i][j] > 0:
                        error_ij = ratings[i][j] - np.dot(self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :]), self.item_matrix[j, :])

                        # Update user and item matrices
                        self.user_matrix[i, :] += self.beta * (2 * error_ij * self.item_matrix[j, :] - self.alpha * self.user_matrix[i, :])
                        self.item_matrix[j, :] += self.beta * (2 * error_ij * (self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) - self.alpha * self.item_matrix[j, :])

                        # Update user and item feature matrices
                        self.user_feature_matrix[i, :] += self.beta * (2 * error_ij * self.item_matrix[j, :] * self.item_feature_matrix[j, :] - self.alpha * self.user_feature_matrix[i, :])
                        self.item_feature_matrix[j, :] += self.beta * (2 * error_ij * (self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) * self.user_feature_matrix[i, :] - self.alpha * self.item_feature_matrix[j, :])

        # Predict ratings for all user-item pairs
        predicted_ratings = np.dot(self.user_matrix + np.dot(self.user_feature_matrix, self.item_feature_matrix.T), self.item_matrix.T)

        return predicted_ratings


In [ ]:
#SVD model with additional user and item features

class SVDModel:
    def __init__(self, num_users, num_items, num_user_features, num_item_features, latent_dim=10, alpha=0.01, beta=0.01, iterations=1000):
        self.num_users = num_users
        self.num_items = num_items
        self.num_user_features = num_user_features
        self.num_item_features = num_item_features
        self.latent_dim = latent_dim
        self.n_factors = n_factors
        self.reg_all = reg_all
        self.alpha = alpha  # Regularization term for user and item matrices
        self.beta = beta    # Learning rate
        self.iterations = iterations

    def fit(self, ratings, user_features, item_features):
        # Initialize user and item matrices
        self.user_matrix = np.random.rand(self.num_users, self.latent_dim)
        self.item_matrix = np.random.rand(self.num_items, self.latent_dim)

        # Initialize user and item feature matrices
        self.user_feature_matrix = np.random.rand(self.num_users, self.num_user_features)
        self.item_feature_matrix = np.random.rand(self.num_items, self.num_item_features)

        # Gradient descent for SVD model with features
        for _ in range(self.iterations):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if ratings[i][j] > 0:
                        error_ij = ratings[i][j] - np.dot(self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :]), self.item_matrix[j, :])
                        for k in range(self.latent_dim):
                            self.user_matrix[i][k] += self.beta * (2 * error_ij * self.item_matrix[j][k] - self.alpha * self.user_matrix[i][k])
                            self.item_matrix[j][k] += self.beta * (2 * error_ij * (self.user_matrix[i][k] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) - self.alpha * self.item_matrix[j][k])

                        for l in range(self.num_user_features):
                            self.user_feature_matrix[i][l] += self.beta * (2 * error_ij * self.item_matrix[j][k] * self.item_feature_matrix[j][l] - self.alpha * self.user_feature_matrix[i][l])

                        for m in range(self.num_item_features):
                            self.item_feature_matrix[j][m] += self.beta * (2 * error_ij * (self.user_matrix[i][k] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) * self.user_feature_matrix[i][k] - self.alpha * self.item_feature_matrix[j][m])

        # Predict ratings for all user-item pairs
        predicted_ratings = np.dot(self.user_matrix + np.dot(self.user_feature_matrix, self.item_feature_matrix.T), self.item_matrix.T)

        return predicted_ratings


In [ ]:
print(ratings_matrix.shape)
print(user_features_matrix.shape)
print(item_features_matrix.shape)

(1132367, 231637)
(1132367, 3)
(231637, 3)


In [ ]:
model = SVDModel(num_users, num_items, num_user_features, num_item_features, n_factors=10, reg_all=0.07)
predicted_ratings = model.fit(ratings_matrix, user_features_matrix, item_features_matrix)

print("Predicted Ratings:")
print(predicted_ratings)


ValueError: operands could not be broadcast together with shapes (3,10) (3,231637) 

In [ ]:
model = SVDModel(num_users, num_items, num_user_features, num_item_features)
predicted_ratings = model.fit(ratings_matrix, user_features_matrix, item_features_matrix)

# Make predictions (for example, predict ratings for all user-item pairs)
print("Predicted Ratings:")
print(predicted_ratings)


ValueError: operands could not be broadcast together with shapes (3,10) (3,231637) 